In [1]:
import sklearn
sklearn_version = sklearn.__version__
print(sklearn_version)
#needs to be 0.24.2

import gensim
gensim.__version__
#3.8.0

0.24.2


'3.8.0'

In [34]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.deprecated.doc2vec import LabeledSentence
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
from string import digits

import pandas as pd
import numpy as np
import string
import re
import random 
import os
import csv
import pickle

from sklearn import metrics
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, plot_precision_recall_curve, auc, average_precision_score,classification_report, confusion_matrix, accuracy_score, average_precision_score, precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score, cross_validate, RepeatedStratifiedKFold, train_test_split,KFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing


from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from modAL.uncertainty import entropy_sampling
from modAL.density import information_density

from scipy.stats import entropy
from matplotlib import pyplot as plt
%matplotlib inline

from numpy import mean
from numpy import std

# Load Data and Clean

In [3]:
data = pd.read_csv("/Users/carlyknight/Dropbox/PROJECTS/Corporate Annual Reports/Replication Folder RR/dictionary_verification/training_paragraphlevel_8-30-21.csv")
data.shape

(1000, 11)

In [4]:
data.columns

Index(['long_filename', 'doc_id', 'year', 'text', 'coop_count', 'coord_count',
       'fulltext_count', 'coord_score', 'coop_score', 'Coordination',
       'Cooperation'],
      dtype='object')

In [5]:
# preprocessing (remove punctuation, numbers, lowercase)/ remove stopwords
punctuation_dictionary = {s:None for s in list(string.punctuation)}

punctuation_translator = str.maketrans(punctuation_dictionary)

def text_cleaner(text, punctuation_translator):
    text=str(text)
    text = text.replace('c("', '')
    text = str(text).translate(punctuation_translator)
    text = text.lower()
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    text= remove_stopwords(text)
    return(text)

data["clean_text"] = data["text"].apply(lambda x: text_cleaner(x, punctuation_translator))

In [6]:
# find phrases
phrases1 = Phrases(map(lambda x: x.split(), data["clean_text"].tolist())) #bigram
phrases2 = Phrases(phrases1[map(lambda x: x.split(), data["clean_text"].tolist())]) #trigram
data["phrased_text"] = data["clean_text"].apply(lambda x: " ".join(phrases2[phrases1[x.split()]]))

#add row_id
data['row.id'] = np.arange(len(data))
data["phrased_text"].head

<bound method NDFrame.head of 0      atyred checchxcxac cochairman gary l wilson co...
1      management took actions year reduce costs cons...
2      finest fsolntcs gii iced service icljxcxac sei...
3      union_camp aggressively pursued forest managem...
4      stockholder rights plan pursuant stockholder r...
                             ...                        
995    community relations highlightsat spokane washi...
996    past_years time unprecedented worldwide growth...
997    fenix union_pacific long forefront developing ...
998    note pension retirement_plans company_subsidia...
999    safe harbor statement nature companys operatio...
Name: phrased_text, Length: 1000, dtype: object>

# Combine cooperation & coordination

In [7]:
#drop data that has both coordination and cooperation
data_multi = data[~((data["Cooperation"] == 1) & (data["Coordination"] == 1))]


In [8]:
#create an multinomial outcome
data_multi['Outcome']=0
data_multi.loc[data_multi['Cooperation'] ==1, 'Outcome'] = 1
data_multi.loc[data_multi['Coordination'] ==1, 'Outcome'] = 2

/Users/carlyknight/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [9]:
data_multi['Outcome'].value_counts()

0    588
1    201
2    190
Name: Outcome, dtype: int64

In [10]:
#vectorization
vectorizer = CountVectorizer(min_df=5)
X = vectorizer.fit_transform(data_multi["phrased_text"]).toarray()

tfidfconverter = TfidfTransformer()
X_tf = tfidfconverter.fit_transform(X).toarray()

y = np.array(data_multi['Outcome'])

In [22]:
#training set and scale
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.3)

In [33]:

model = LogisticRegression(multi_class='multinomial', solver = 'lbfgs')
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X_train, y_train, scoring='recall', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

TypeError: 'numpy.float64' object is not callable

In [37]:
#print(metrics.classification_report(y_true, y_pred, digits=3))

In [37]:
#hyperparameters
#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

#training set
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.3)

model = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model.predict(X_test)

#print the tunable parameters (They were not tuned in this example, everything kept as default)
params = model.get_params()
print(params)

confusion_matrix(y_test, preds)
#transform confusion matrix into array
confmtrx = np.array(confusion_matrix(y_test, preds))

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'multinomial', 'n_jobs': None, 'penalty': 'none', 'random_state': 0, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [38]:
#Create DataFrame from confmtrx array 
#rows for test:
#columns for preds: male, predicted_female, predicted_infant as column

pd.DataFrame(confmtrx, index=['None','Cooperation', 'Coordination'],
columns=['predicted_none', 'predicted_cooperation', 'predicted_coordination'])


,predicted_none,predicted_cooperation,predicted_coordination
None,149,8,20
Cooperation,29,25,6
Coordination,27,6,24


In [40]:
#Create classification report
class_report=classification_report(y_test, preds)
print(class_report)

              precision    recall  f1-score   support

           0       0.73      0.84      0.78       177
           1       0.64      0.42      0.51        60
           2       0.48      0.42      0.45        57

    accuracy                           0.67       294
   macro avg       0.62      0.56      0.58       294
weighted avg       0.66      0.67      0.66       294



# Logistic: Cooperation

#https://medium.com/analytics-vidhya/applying-text-classification-using-logistic-regression-a-comparison-between-bow-and-tf-idf-1f1ed1b83640

#best preprocessing seems to be: remove digits, remove punctuation, do not lowercase,do not remove stopwords

#https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/

In [7]:
#vectorization
vectorizer = CountVectorizer(min_df=5)
X = vectorizer.fit_transform(data["phrased_text"]).toarray()

tfidfconverter = TfidfTransformer()
X_tf = tfidfconverter.fit_transform(X).toarray()

y = np.array(data['Cooperation'])

In [8]:
#hyperparameters
#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

#training set
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.3)

# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.05, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search_coop = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='precision',error_score=0)
grid_result_coop = grid_search_coop.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result_coop.best_score_, grid_result_coop.best_params_))
means = grid_result_coop.cv_results_['mean_test_score']
stds = grid_result_coop.cv_results_['std_test_score']
params = grid_result_coop.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.666746 using {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.584601 (0.150546) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.584601 (0.150546) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.585342 (0.151546) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.666746 (0.200054) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.666746 (0.200054) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.666746 (0.200054) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.333333 (0.471405) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.333333 (0.471405) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.333333 (0.471405) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.000000 (0.000000) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with

In [9]:
y_pred = grid_result_coop.best_estimator_.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))

Accuracy:  0.8066666666666666
Precision:  0.9047619047619048
Recall:  0.25333333333333335


In [10]:
#save
import joblib
joblib.dump(grid_search_coop.best_estimator_, "/Users/carlyknight/Dropbox/PROJECTS/Corporate Annual Reports/Replication Folder RR/coord_coop/cooperation_logistic.pkl")

['/Users/carlyknight/Dropbox/PROJECTS/Corporate Annual Reports/Replication Folder RR/coord_coop/cooperation_logistic.pkl']

# Logistic: Coordination

#https://medium.com/analytics-vidhya/applying-text-classification-using-logistic-regression-a-comparison-between-bow-and-tf-idf-1f1ed1b83640

#best preprocessing seems to be: remove digits, remove punctuation, do not lowercase,do not remove stopwords

#https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/

In [11]:
y = np.array(data['Coordination'])

In [12]:
#hyperparameters
#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

#training set
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.3)

# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.05, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search_coord = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='precision',error_score=0)
grid_result_coord = grid_search_coord.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result_coord.best_score_, grid_result_coord.best_params_))
means = grid_result_coord.cv_results_['mean_test_score']
stds = grid_result_coord.cv_results_['std_test_score']
params = grid_result_coord.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.616987 using {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.543870 (0.154505) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.543870 (0.154505) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.543870 (0.154505) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.616987 (0.198438) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.616987 (0.198438) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.610717 (0.190944) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.100000 (0.300000) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.100000 (0.300000) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.100000 (0.300000) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.000000 (0.000000) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with

In [13]:
y_pred = grid_search_coord.best_estimator_.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))

Accuracy:  0.8
Precision:  0.6086956521739131
Recall:  0.2153846153846154


### Make predictions on entire dataset

In [14]:
#open all data
files = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/wagefirmtexts/wagefirmtexts_afterML/by_para/"
wagefiles = os.listdir(files)
               

In [15]:
#create list of paragraphs
paragraph_list = []

for file in wagefiles:
    #open text
    with open(os.path.join(files, str(file)),  'rb') as f:
        text = f.readlines()
        paragraph_list.append([file, text])

In [16]:
len(paragraph_list)
#paragraph should be around 88,000

88521

In [17]:
#turn into pandas
df = pd.DataFrame.from_records(paragraph_list)
df.columns = ['doc.id', 'para']

#add row_id
df['row.id'] = np.arange(len(df))

In [18]:
df.shape

(88521, 3)

In [19]:
# preprocessing (remove punctuation, numbers, lowercase)/ remove stopwords
punctuation_dictionary = {s:None for s in list(string.punctuation)}

punctuation_translator = str.maketrans(punctuation_dictionary)


def text_cleaner(text, punctuation_translator):
    text=str(text)
    text = text.replace('c("', '')
    text = str(text).translate(punctuation_translator)
    text = text.lower()
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    text= remove_stopwords(text)
    return(text)

df["clean_para"] = df["para"].apply(lambda x: text_cleaner(x, punctuation_translator))

In [ ]:
#phrases
phrases1 = Phrases(map(lambda x: x.split(), df["clean_para"].tolist())) #bigram
phrases2 = Phrases(phrases1[map(lambda x: x.split(), df["clean_para"].tolist())]) #trigram
df["clean_phrase"] = df["clean_para"].apply(lambda x: " ".join(phrases2[phrases1[x.split()]]))

In [20]:
#vectorization
X = vectorizer.transform(df["clean_para"]).toarray()
X_tf = tfidfconverter.transform(X).toarray()

In [21]:
#Cooperation predictions
cooperation = grid_search_coop.best_estimator_.predict(X_tf)

#Coordination predictions
coordination = grid_search_coord.best_estimator_.predict(X_tf)

In [22]:
#put predictions in df
df['coop_hat'] = cooperation
df['coord_hat'] = coordination

df.to_csv("/Users/carlyknight/Dropbox/PROJECTS/Corporate Annual Reports/Replication Folder RR/coord_coop/coord_coop_scores_afterml.csv")

In [23]:
sum(df['coop_hat'])
#7,525

7525

In [24]:
sum(df['coord_hat'])
#7,631

7631